# IMT 573 - Problem Set 4 - Working With Data Part 2

### Instructions

Before beginning this assignment, please ensure you have access to a working instance of Jupyter Notebooks with Python 3.

1. First, replace the “YOUR NAME HERE” text in the next cell with your own full name. Any collaborators must also be listed in this cell.

2. Be sure to include well-documented (e.g. commented) code cells, figures, and clearly written text  explanations as necessary. Any figures should be clearly labeled and appropriately referenced within the text. Be sure that each visualization adds value to your written explanation; avoid redundancy – you do no need four different visualizations of the same pattern.

3. Collaboration on problem sets and labs is fun, useful, and encouraged. However, each student must turn in an individual write-up in their own words as well as code/work that is their own. Regardless of whether you work with others, what you turn in must be your own work; this includes code and interpretation of results. The names of all collaborators must be listed on each assignment. Do not copy-and-paste from other students’ responses or code - your code should never be on any other student's screen or machine.

4. All materials and resources that you use (with the exception of lecture slides) must be appropriately referenced within your assignment.

5. Partial credit will be awarded for each question for which a serious attempt at finding an answer has been shown. Students are *strongly* encouraged to attempt each question and document their reasoning process even if they cannot find the correct answer. 

6. After completing the assignment, ensure that your code can run from start to finish without issue. Restart the kernal and run all cells to double check.

Name: Pratiibh Bassi

Collaborators: 

For this assignment, you'll need (at least) the following packages. If the package does not load, be sure it is properly installed.

In [5]:
import pandas as pd
import censusgeocode as cg
import seaborn as sns
import matplotlib.pyplot as plt

In this problem set, we will be joining disparate sets of data - namely: Seattle police crime data, information on Seattle police beats, and education attainment from the US Census. Our goal is to build a dataset where we can examine questions around crimes in Seattle and the educational attainment of people living in the areas in which the crime occurred; this requires data
to be combined from these multiple individual sources.

As a general rule, be sure to keep copies of the original dataset(s) as you work through cleaning (remember
data provenance!).

### Problem 1: Crime Data

Load the Seattle crime data from the provided `crime_data.csv` data file. The data is a modified version of the data available [here](https://data.seattle.gov/Public-Safety/Crime-Data/4fs7-3vj5). We will call this dataset the “Crime Dataset” going forward.

In [9]:
crime_data = pd.read_csv("../Downloads/Crime_Data.csv")

#### (a) Basic inspection

Perform a basic inspection of the Crime Dataset and discuss what you find. How many observations are there? Is there any anomolous or missing data?

In [11]:
crime_data.head()

,Report Number,Occurred Date,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood
0,1975000079415,12/16/1975,900.0,12/16/1975,1500.0,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,SOUTH,R,R3,LAKEWOOD/SEWARD PARK
1,1976000069169,01/01/1976,1.0,01/31/1976,2359.0,SEX OFFENSE-OTHER,SEXOFF-INDECENT LIBERTIES,UNKNOWN,NaN,NaN,UNKNOWN
2,1979000049118,01/28/1979,1600.0,02/09/1979,1430.0,CAR PROWL,THEFT-CARPROWL,EAST,G,G2,CENTRAL AREA/SQUIRE PARK
3,19810000305573,08/22/1981,2029.0,08/22/1981,2030.0,HOMICIDE,HOMICIDE-PREMEDITATED-WEAPON,SOUTH,S,S2,BRIGHTON/DUNLAP
4,1981000076456,02/14/1981,2000.0,02/15/1981,435.0,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,SOUTHWEST,W,W3,ROXHILL/WESTWOOD/ARBOR HEIGHTS


In [13]:
crime_data.tail()

,Report Number,Occurred Date,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood
523586,2019000099944,03/20/2019,1713.0,03/20/2019,1713.0,FAMILY OFFENSE-NONVIOLENT,CHILD-OTHER,SOUTH,O,O3,MID BEACON HILL
523587,2019000099946,03/20/2019,730.0,03/20/2019,1721.0,BURGLARY-RESIDENTIAL,BURGLARY-FORCE-RES,EAST,C,C2,MONTLAKE/PORTAGE BAY
523588,2019000099949,03/20/2019,1724.0,03/20/2019,1724.0,ROBBERY-COMMERCIAL,ROBBERY-BUSINESS-BODYFORCE,SOUTH,S,S2,RAINIER BEACH
523589,2019000099974,03/20/2019,1750.0,03/20/2019,1904.0,THEFT-SHOPLIFT,THEFT-SHOPLIFT,NORTH,L,L2,NORTHGATE
523590,2019000099993,03/19/2019,1800.0,03/20/2019,2237.0,THEFT-ALL OTHER,THEFT-OTH,NORTH,N,N1,BITTERLAKE


In [15]:
crime_data.describe()

,Report Number,Occurred Time,Reported Time
count,5.235910e+05,523589.000000,523589.000000
mean,1.634634e+13,1358.647834,1353.365256
std,7.421091e+12,688.350593,589.365021
min,2.008000e+08,0.000000,0.000000
25%,2.008000e+13,900.000000,950.000000
50%,2.012000e+13,1500.000000,1407.000000
75%,2.016000e+13,1920.000000,1817.000000
max,2.019000e+13,2359.000000,2359.000000


In [19]:
crime_data.shape

(523591, 11)

In [21]:
crime_data.isna().sum(axis = 0)

Report Number                     0
Occurred Date                     2
Occurred Time                     2
Reported Date                     0
Reported Time                     2
Crime Subcategory               262
Primary Offense Description       0
Precinct                          6
Sector                         3346
Beat                           3298
Neighborhood                      0
dtype: int64

#### (b) Years of crimes

Examine the years in which crimes were committed in the dataset. What is the earliest year in the dataset? Are there any distinct trends with the annual number of crimes committed in the dataset?

In [23]:
filtered_crime_data = crime_data.copy()

In [25]:
filtered_crime_data['Year'] = filtered_crime_data['Occurred Date'].str.slice(6,10)
filtered_crime_data['Year']

0         1975
1         1976
2         1979
3         1981
4         1981
          ... 
523586    2019
523587    2019
523588    2019
523589    2019
523590    2019
Name: Year, Length: 523591, dtype: object

In [27]:
filtered_crime_data['Year'].value_counts(dropna = False)

Year
2018    51302
2017    50334
2014    49322
2016    49220
2015    47693
2013    45551
2009    45056
2010    43353
2008    42793
2011    41298
2012    41007
2019    15624
2007      627
2006      100
2001       53
2005       49
2004       43
2000       40
2003       29
1998       20
2002       17
1999        8
1995        6
1993        5
1997        5
1996        4
1991        4
1976        2
1979        2
1981        2
1985        2
1994        2
1975        2
NaN         2
1990        2
1980        2
1973        1
1989        1
1987        1
1964        1
1988        1
1986        1
1908        1
1978        1
1977        1
1974        1
Name: count, dtype: int64

So, the earliest year in the dataset is 1908.  I notice that the consistency of records being added happens after 2008, and years before that, it wasn't really consistently adding records in.

#### (c) Filter data on years

Subset the data to only include crimes that were committed after 2011 (remember good practices of data
provenance!). Going forward, we will use this data subset. Print the shape of your dataset after doing this.

In [30]:
filtered_crime_data = filtered_crime_data[filtered_crime_data.Year > '2011']

In [32]:
filtered_crime_data.shape

(350053, 12)

#### (d) Looking at frequency of beats 

Each of the records has a police beat associated with it. You can learn more about police beats [here](https://www.seattle.gov/police/information-and-data/data/tweets-by-beat). How frequently are the beats in the (filtered) Crime Dataset listed? Are there any anomolies with how frequently some of the beats are listed? Are there missing beats?

In [34]:
filtered_crime_data['Beat'].value_counts()

Beat
K3     11611
M2     10210
E2     10200
U1     10157
L2     10049
M1      9883
M3      9723
B2      9253
Q3      9249
U3      9019
U2      8866
B3      8846
Q2      8159
D1      8066
B1      7954
N3      7517
D2      7491
E1      7459
R2      7448
N2      7409
J3      7203
E3      7032
R3      6909
K1      6611
J2      6585
K2      6560
D3      6530
W2      6514
F2      6429
R1      6080
S3      6027
L1      5823
L3      5710
C1      5694
J1      5668
Q1      5647
F3      5361
N1      5303
W3      5286
G2      5259
S2      5139
W1      5135
S1      4819
C2      4789
C3      4726
O1      4523
F1      4332
G3      4327
G1      3257
O3      3239
O2      2894
DET        7
S          4
W          3
SS         1
CTY        1
WS         1
N          1
K          1
Name: count, dtype: int64

In [36]:
filtered_crime_data.isnull().sum()

Report Number                     0
Occurred Date                     0
Occurred Time                     0
Reported Date                     0
Reported Time                     0
Crime Subcategory                 0
Primary Offense Description       0
Precinct                          3
Sector                         2071
Beat                           2054
Neighborhood                      0
Year                              0
dtype: int64

In [38]:
print(filtered_crime_data['Year'].unique())

print(filtered_crime_data.groupby('Year').size())

['2012' '2013' '2014' '2015' '2016' '2017' '2018' '2019']
Year
2012    41007
2013    45551
2014    49322
2015    47693
2016    49220
2017    50334
2018    51302
2019    15624
dtype: int64


In [40]:
filtered_crime_data_missing_beats = filtered_crime_data[filtered_crime_data['Beat'].isnull()]

print( (filtered_crime_data_missing_beats.groupby('Year').size()/filtered_crime_data.groupby('Year').size()) * 100 )

Year
2012    0.565757
2013    0.638844
2014    0.444021
2015    0.421445
2016    0.530272
2017    0.737076
2018    0.736813
2019    0.646441
dtype: float64


So, there are missing beats as we can see from filtered_crime_data.isnull().sum() statement.  There are 2054 missing beat reports.  Some anomalies are how there are weird beats with only 1 letter or 2 letters that stand out from the normal letter + number format used for Beats.  Most of the beats have several thousands of entries, but when they don't it is always below 10, which is interestingly weird.

### Problem 2: Police Beat Data

Load the data on Seattle police beats provided in the police_beat_and_precinct_centerpoints.csv. This is the same data that you used in Lab 4. You can learn more about police precincts and beats [here](https://www.seattle.gov/police/about-us/about-policing/precinct-and-patrol-boundaries). The data can be found in the `Police_Beat_and_Precinct_Centerpoints.csv` file.

In [43]:
beat_data = pd.read_csv("../Downloads/Police_Beat_and_Precinct_Centerpoints.csv")


#### (a) Missing beats

Does the (filtered) Crime Dataset include police beats that are not present in the Beats Dataset? If so, how many and
with what frequency do they occur? Would you say that these comprise a large number of the observations
in the Crime Dataset or are they rather infrequent? Do you think removing them would drastically alter the
scope of the Crime Dataset?

In [45]:
beat_data.head()

,Name,Location 1,Latitude,Longitude
0,B1,"(47.7097756394592, -122.370990523069)",47.70978,-122.37099
1,B2,"(47.6790521901374, -122.391748391741)",47.67905,-122.39175
2,B3,"(47.6812920482227, -122.364236159741)",47.68129,-122.36424
3,C1,"(47.6342500180223, -122.315684762418)",47.63425,-122.31568
4,C2,"(47.6192385752996, -122.313557430551)",47.61924,-122.31356


In [47]:
beat_data.shape

(57, 4)

In [53]:
beat_data_beats = set(beat_data.Name)
beat_data_beats

{'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'CITYWIDE',
 'D1',
 'D2',
 'D3',
 'E',
 'E1',
 'E2',
 'E3',
 'F1',
 'F2',
 'F3',
 'G1',
 'G2',
 'G3',
 'J1',
 'J2',
 'J3',
 'K1',
 'K2',
 'K3',
 'L1',
 'L2',
 'L3',
 'M1',
 'M2',
 'M3',
 'N',
 'N1',
 'N2',
 'N3',
 'O1',
 'O2',
 'O3',
 'Q1',
 'Q2',
 'Q3',
 'R1',
 'R2',
 'R3',
 'S1',
 'S2',
 'S3',
 'SE',
 'SW',
 'U1',
 'U2',
 'U3',
 'W',
 'W1',
 'W2',
 'W3'}

In [55]:
crime_data_beats = set(filtered_crime_data.Beat)
crime_data_beats

{'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'CTY',
 'D1',
 'D2',
 'D3',
 'DET',
 'E1',
 'E2',
 'E3',
 'F1',
 'F2',
 'F3',
 'G1',
 'G2',
 'G3',
 'J1',
 'J2',
 'J3',
 'K',
 'K1',
 'K2',
 'K3',
 'L1',
 'L2',
 'L3',
 'M1',
 'M2',
 'M3',
 'N',
 'N1',
 'N2',
 'N3',
 'O1',
 'O2',
 'O3',
 'Q1',
 'Q2',
 'Q3',
 'R1',
 'R2',
 'R3',
 'S',
 'S1',
 'S2',
 'S3',
 'SS',
 'U1',
 'U2',
 'U3',
 'W',
 'W1',
 'W2',
 'W3',
 'WS',
 nan}

In [57]:
crime_data_beats - beat_data_beats

{'CTY', 'DET', 'K', 'S', 'SS', 'WS', nan}

In [59]:
filtered_crime_data.Beat.isin(list(crime_data_beats - beat_data_beats))

172763    False
172764    False
172765    False
172766    False
172767    False
          ...  
523586    False
523587    False
523588    False
523589    False
523590    False
Name: Beat, Length: 350053, dtype: bool

In [61]:
filtered_crime_data[filtered_crime_data.Beat.isin(list(crime_data_beats - beat_data_beats))]

,Report Number,Occurred Date,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood,Year
172809,20120000100551,04/02/2012,2300.0,04/04/2012,1305.0,NARCOTIC,NARC-POSSESS-COCAINE,UNKNOWN,NaN,NaN,UNKNOWN,2012
172915,20120000101769,03/01/2012,1200.0,04/04/2012,1536.0,SEX OFFENSE-OTHER,SEXOFF-OTHER,UNKNOWN,NaN,NaN,UNKNOWN,2012
173067,2012000010329,01/10/2012,2345.0,01/10/2012,2345.0,NARCOTIC,NARC-POSSESS-METH,UNKNOWN,NaN,NaN,UNKNOWN,2012
173261,20120000105575,04/06/2012,0.0,04/07/2012,1256.0,ROBBERY-STREET,ROBBERY-STREET-BODYFORCE,UNKNOWN,NaN,NaN,UNKNOWN,2012
173279,20120000105887,04/06/2012,1530.0,04/07/2012,1904.0,CAR PROWL,THEFT-CARPROWL,UNKNOWN,NaN,NaN,UNKNOWN,2012
...,...,...,...,...,...,...,...,...,...,...,...,...
522730,20190000904938,05/01/2019,1200.0,05/03/2019,845.0,THEFT-ALL OTHER,THEFT-OTH,UNKNOWN,NaN,NaN,UNKNOWN,2019
522733,20190000904942,05/03/2019,1310.0,05/03/2019,2316.0,THEFT-ALL OTHER,THEFT-OTH,UNKNOWN,NaN,NaN,UNKNOWN,2019
522917,2019000091643,03/14/2019,1539.0,03/14/2019,1539.0,TRESPASS,TRESPASS,UNKNOWN,NaN,NaN,UNKNOWN,2019
523223,2019000095610,03/16/2019,2300.0,03/17/2019,1407.0,THEFT-ALL OTHER,THEFT-PKPOCKET,UNKNOWN,NaN,NaN,UNKNOWN,2019


In [63]:
filtered_crime_data[filtered_crime_data.Beat.isin(list(crime_data_beats - beat_data_beats))].Beat.value_counts(dropna = False)

Beat
NaN    2054
DET       7
S         4
SS        1
CTY       1
WS        1
K         1
Name: count, dtype: int64

If we don't count missing/NA values, there are 6 beats which aren't present in the Beats Dataset.  These are infrequent and removing them shouldn't affect future analysis much if at all.

#### (b) Filtering beats

Let’s remove all instances in the (filtered) Crime Dataset that have beats which occur fewer than 10 times the dataset. Also remove any observations with missing beats. After only keeping years of interest and filtering based on frequency of the beat, how many observations do we now have in the Crime Dataset?

In [68]:
counts = filtered_crime_data.Beat.value_counts()
counts = counts[counts >= 10]
counts.head()


Beat
K3    11611
M2    10210
E2    10200
U1    10157
L2    10049
Name: count, dtype: int64

In [70]:
filtered_crime_data = filtered_crime_data[~filtered_crime_data.Beat.isna()]

In [72]:
filtered_crime_data.head()

,Report Number,Occurred Date,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood,Year
172763,20120000100012,04/02/2012,2040.0,04/03/2012,28.0,NARCOTIC,NARC-POSSESS-MARIJU,WEST,K,K2,PIONEER SQUARE,2012
172764,20120000100035,04/02/2012,2100.0,04/02/2012,2103.0,ROBBERY-COMMERCIAL,ROBBERY-BUSINESS-GUN,NORTH,B,B2,BALLARD SOUTH,2012
172765,20120000100055,04/02/2012,1930.0,04/02/2012,2126.0,MOTOR VEHICLE THEFT,VEH-THEFT-AUTO,NORTH,J,J1,BALLARD NORTH,2012
172766,20120000100062,04/02/2012,2144.0,04/02/2012,2144.0,DUI,DUI-LIQUOR,EAST,E,E3,CAPITOL HILL,2012
172767,20120000100092,04/02/2012,2218.0,04/02/2012,2218.0,ROBBERY-RESIDENTIAL,ROBBERY-RESIDENCE-BODYFORCE,EAST,C,C2,MADISON PARK,2012


In [74]:
filtered_crime_data.shape

(347999, 12)

There are 347999 observations across 12 variables in the Crime Dataset.

#### (c) Filtering beats

To join the Beat Dataset to census data, we must have census tract information. In a previous lab, you used an API to get the census information for each police beat. Use the code from that lab to add a column named "tract" to the Beats Dataset. Remember good practices of data provenance!

In [77]:
def get_census_tract(long_in, lat_in):
    return cg.coordinates(long_in, lat_in)['Census Tracts'][0]['GEOID']

In [79]:
tracts = []
for long, lat in zip(beat_data.Longitude, beat_data.Latitude):
    tracts.append(get_census_tract(long, lat))

In [81]:
beat_data_copy = beat_data.copy()
beat_data_copy['tract'] = tracts

In [83]:
beat_data_copy.head()

,Name,Location 1,Latitude,Longitude,tract
0,B1,"(47.7097756394592, -122.370990523069)",47.70978,-122.37099,53033001400
1,B2,"(47.6790521901374, -122.391748391741)",47.67905,-122.39175,53033003202
2,B3,"(47.6812920482227, -122.364236159741)",47.68129,-122.36424,53033002900
3,C1,"(47.6342500180223, -122.315684762418)",47.63425,-122.31568,53033006500
4,C2,"(47.6192385752996, -122.313557430551)",47.61924,-122.31356,53033007502


#### (d) Process for joining

We will eventually join the Beats Dataset to the Crime Dataset. We could have joined the two and then
found the census tracts for each beat. Would there have been a particular advantage/disadvantage to doing this join first and then finding census tracts? If so, what is it? (NOTE: you do not need to write any code to answer this)

There's a distinct disadvantage to performing the join first.  The crime dataset contains far more observations than the Beats dataset.  The beats in the crime dataset are also not unique, so there are thousands of observations for each beat. Performing the geolocation above is computationally expensive and if we were to iterate through the crime dataset, we would be performing this expensive computation across rows with non-unique beat values.  Thus, performing the geolocation on unique list of beats in the beats dataset and then using a join operation later is much more computationally viable.

#### (e) Extracting 11-digit codes 
The census data uses an 11-digit code that consists of the state, county, and tract code. It does not include the block code. To join the census data to the Beats Dataset, we must have this code for each of the beats. Extract the 11-digit code for each of the beats in the Beats Dataset. The 11 digits consist of the 2 state digits, 3 county digits, and 6 tract digits. Add a column to the (modified) Beats Dataset named `census_id` with the 11-digit code for each beat. Ensure that the values in the `census_id` column are stored as `int64`.

You can learn more about the codes [here](https://transition.fcc.gov/form477/Geo/more_about_census_blocks.pdf). 

In [86]:
beat_data_copy.tract.head()


0    53033001400
1    53033003202
2    53033002900
3    53033006500
4    53033007502
Name: tract, dtype: object

In [88]:
beat_data_copy['census_id'] = beat_data_copy.tract.str.slice(0,11).astype('Int64')

beat_data_copy.head()

,Name,Location 1,Latitude,Longitude,tract,census_id
0,B1,"(47.7097756394592, -122.370990523069)",47.70978,-122.37099,53033001400,53033001400
1,B2,"(47.6790521901374, -122.391748391741)",47.67905,-122.39175,53033003202,53033003202
2,B3,"(47.6812920482227, -122.364236159741)",47.68129,-122.36424,53033002900,53033002900
3,C1,"(47.6342500180223, -122.315684762418)",47.63425,-122.31568,53033006500,53033006500
4,C2,"(47.6192385752996, -122.313557430551)",47.61924,-122.31356,53033007502,53033007502


#### (f) Extracting 11-digit codes from census 

Now, we will examine Census Dataset in the `census_data_2020_edu_attainment.csv` file. The data includes counts of education attainment across different census tracts. Note how this data is in a "wide" format and how it can be converted to a "long" format. For now, we will work with it as is. 

The census data contains a `GEO_ID` column. Among other things, this variable encodes the 11-digit code that we had extracted above for each of the police beats. Specifically, when we look at the characters after the characters "US" for values of GEO.id, we see encodings for state, county, and tract, which should align with the beats we had above. Extract the 11-digit code from the `GEO_ID` column. Add a column named `census_id` to the Census Dataset with the 11-digit code for each census observation. Be sure to practice good data provenance when modifying a dataset.

In [90]:
census_data = pd.read_csv('../Downloads/census_data_2020_edu_attainment.csv')

In [92]:
census_data.head()

,GEO_ID,Population 18 to 24 years,Population 18 to 24 years-Less than high school graduate,Population 18 to 24 years-High school graduate (includes equivalency),Population 18 to 24 years-Some college or associate's degree,Population 18 to 24 years-Bachelor's degree or higher,Population 25 years and over,Population 25 years and over-Less than 9th grade,"Population 25 years and over-9th to 12th grade, no diploma",Population 25 years and over-High school graduate (includes equivalency),"Population 25 years and over-Some college, no degree",Population 25 years and over-Associate's degree,Population 25 years and over-Bachelor's degree,Population 25 years and over-Graduate or professional degree,Population 25 years and over-High school graduate or higher,Population 25 years and over-Bachelor's degree or higher
0,1400000US53033000101,278,71,17,90,100,2718,187,80,492,362,250,949,398,2451,1347
1,1400000US53033000102,402,50,125,104,123,3578,75,172,717,379,315,981,939,3331,1920
2,1400000US53033000201,301,0,234,53,14,3616,15,232,412,512,199,1242,1004,3369,2246
3,1400000US53033000202,228,0,54,102,72,3194,69,187,443,406,179,1174,736,2938,1910
4,1400000US53033000300,477,25,60,318,74,1933,41,113,189,342,223,639,386,1779,1025


In [95]:
census_data_copy = census_data.copy()
census_data_copy['census_id'] = census_data_copy['GEO_ID'].str.slice(9, 20).astype('Int64')
census_data_copy['census_id']

0      53033000101
1      53033000102
2      53033000201
3      53033000202
4      53033000300
          ...     
490    53033032704
491    53033032705
492    53033032706
493    53033032800
494    53033990100
Name: census_id, Length: 495, dtype: Int64

### Problem 3: Census Data

#### (a) Join datasets part 1

Join the Census data in the with the (modified) Beat Dataset using the 11-digit codes as keys. Be sure that you do not lose any of the police beats when doing this join (i.e. your output dataframe should have the same number of rows as the cleaned Beats Dataset - use the correct join). Are there any police beats that do not have any associated census data? If so, how many? Save the resulting data in a different variable (i.e. don't overwrite the Beats or Census Datasets).

In [97]:
joined_data = beat_data_copy.merge(census_data_copy, on = 'census_id', how = "left")


In [99]:
joined_data.shape

(57, 22)

In [101]:
joined_data.head()

,Name,Location 1,Latitude,Longitude,tract,census_id,GEO_ID,Population 18 to 24 years,Population 18 to 24 years-Less than high school graduate,Population 18 to 24 years-High school graduate (includes equivalency),...,Population 25 years and over,Population 25 years and over-Less than 9th grade,"Population 25 years and over-9th to 12th grade, no diploma",Population 25 years and over-High school graduate (includes equivalency),"Population 25 years and over-Some college, no degree",Population 25 years and over-Associate's degree,Population 25 years and over-Bachelor's degree,Population 25 years and over-Graduate or professional degree,Population 25 years and over-High school graduate or higher,Population 25 years and over-Bachelor's degree or higher
0,B1,"(47.7097756394592, -122.370990523069)",47.70978,-122.37099,53033001400,53033001400,1400000US53033001400,83,12,48,...,4352,38,4,646,527,327,1599,1211,4310,2810
1,B2,"(47.6790521901374, -122.391748391741)",47.67905,-122.39175,53033003202,53033003202,1400000US53033003202,181,45,74,...,3656,0,0,209,685,256,1227,1279,3656,2506
2,B3,"(47.6812920482227, -122.364236159741)",47.68129,-122.36424,53033002900,53033002900,1400000US53033002900,205,56,50,...,3597,0,38,312,655,92,1461,1039,3559,2500
3,C1,"(47.6342500180223, -122.315684762418)",47.63425,-122.31568,53033006500,53033006500,1400000US53033006500,302,9,13,...,4125,0,50,153,395,177,1923,1427,4075,3350
4,C2,"(47.6192385752996, -122.313557430551)",47.61924,-122.31356,53033007502,53033007502,1400000US53033007502,322,34,0,...,2247,0,0,43,293,130,1224,557,2247,1781


In [103]:
joined_data[joined_data.GEO_ID.isna()]

,Name,Location 1,Latitude,Longitude,tract,census_id,GEO_ID,Population 18 to 24 years,Population 18 to 24 years-Less than high school graduate,Population 18 to 24 years-High school graduate (includes equivalency),...,Population 25 years and over,Population 25 years and over-Less than 9th grade,"Population 25 years and over-9th to 12th grade, no diploma",Population 25 years and over-High school graduate (includes equivalency),"Population 25 years and over-Some college, no degree",Population 25 years and over-Associate's degree,Population 25 years and over-Bachelor's degree,Population 25 years and over-Graduate or professional degree,Population 25 years and over-High school graduate or higher,Population 25 years and over-Bachelor's degree or higher


There were no police beats lost during the join operation, and all police beats have data from the census in the joined dataset.

#### (b) Join datasets part 2

Then, join the Crime Dataset to our joined beat/census data. We can do this using the police beat name. Again, be sure you do not lose any observations from the (filtered) Crime Dataset. What are the final dimensions of the joined dataset? Save this joined dataset as you'll use it in a future problem set.

In [106]:
filtered_crime_data.head()


,Report Number,Occurred Date,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood,Year
172763,20120000100012,04/02/2012,2040.0,04/03/2012,28.0,NARCOTIC,NARC-POSSESS-MARIJU,WEST,K,K2,PIONEER SQUARE,2012
172764,20120000100035,04/02/2012,2100.0,04/02/2012,2103.0,ROBBERY-COMMERCIAL,ROBBERY-BUSINESS-GUN,NORTH,B,B2,BALLARD SOUTH,2012
172765,20120000100055,04/02/2012,1930.0,04/02/2012,2126.0,MOTOR VEHICLE THEFT,VEH-THEFT-AUTO,NORTH,J,J1,BALLARD NORTH,2012
172766,20120000100062,04/02/2012,2144.0,04/02/2012,2144.0,DUI,DUI-LIQUOR,EAST,E,E3,CAPITOL HILL,2012
172767,20120000100092,04/02/2012,2218.0,04/02/2012,2218.0,ROBBERY-RESIDENTIAL,ROBBERY-RESIDENCE-BODYFORCE,EAST,C,C2,MADISON PARK,2012


In [108]:
filtered_crime_data.shape

(347999, 12)

In [110]:
joined_data = joined_data.merge(filtered_crime_data, left_on = 'Name', right_on = 'Beat', how = "right")

In [112]:
joined_data.shape

(347999, 34)

In [114]:
joined_data.head()

,Name,Location 1,Latitude,Longitude,tract,census_id,GEO_ID,Population 18 to 24 years,Population 18 to 24 years-Less than high school graduate,Population 18 to 24 years-High school graduate (includes equivalency),...,Occurred Time,Reported Date,Reported Time,Crime Subcategory,Primary Offense Description,Precinct,Sector,Beat,Neighborhood,Year
0,K2,"(47.5998930290529, -122.326813620856)",47.59989,-122.32681,53033009200,53033009200,1400000US53033009200,118.0,0.0,0.0,...,2040.0,04/03/2012,28.0,NARCOTIC,NARC-POSSESS-MARIJU,WEST,K,K2,PIONEER SQUARE,2012
1,B2,"(47.6790521901374, -122.391748391741)",47.67905,-122.39175,53033003202,53033003202,1400000US53033003202,181.0,45.0,74.0,...,2100.0,04/02/2012,2103.0,ROBBERY-COMMERCIAL,ROBBERY-BUSINESS-GUN,NORTH,B,B2,BALLARD SOUTH,2012
2,J1,"(47.676809900774, -122.337899655521)",47.67681,-122.33790,53033004600,53033004600,1400000US53033004600,224.0,34.0,39.0,...,1930.0,04/02/2012,2126.0,MOTOR VEHICLE THEFT,VEH-THEFT-AUTO,NORTH,J,J1,BALLARD NORTH,2012
3,E3,"(47.603162336406, -122.319319689671)",47.60316,-122.31932,53033008600,53033008600,1400000US53033008600,2975.0,44.0,306.0,...,2144.0,04/02/2012,2144.0,DUI,DUI-LIQUOR,EAST,E,E3,CAPITOL HILL,2012
4,C2,"(47.6192385752996, -122.313557430551)",47.61924,-122.31356,53033007502,53033007502,1400000US53033007502,322.0,34.0,0.0,...,2218.0,04/02/2012,2218.0,ROBBERY-RESIDENTIAL,ROBBERY-RESIDENCE-BODYFORCE,EAST,C,C2,MADISON PARK,2012


In [116]:
joined_data.to_csv('joined_census_beat_data.csv', index = False)

The final dimensions of the joined dataset is 347999 rows with 34 columns.